In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, date
import csv
import time
import ssl
import os
import threading
ssl._create_default_https_context = ssl._create_unverified_context
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
DEBUG = False
time_data = [2024, 10, 28]
# time_data = [2021, 8, 11]

In [3]:
output_file_path = r"D:\Stock\daily_result\%s%s%s_trend.csv"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))
# output_file_path = r"D:\Stock\daily_result\%s%s%s_start.csv"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))

In [4]:
def get_twse_stock_db_info():
    link = 'http://www.twse.com.tw/exchangeReport/BWIBBU_ALL?response=open_data'
    df = pd.read_csv(link, encoding='utf_8_sig')
    return df
    
def get_twse_stock_info(df, stock):
    target_data = df[df["股票代號"] == int(stock)]
    name = target_data.iloc[0]['股票名稱']
    priceEarningRatio = target_data.iloc[0]['本益比']
    yieldRatio = target_data.iloc[0]['殖利率(%)']
    priceBookRatio = target_data.iloc[0]['股價淨值比']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_otc_stock_db_info():
    link = 'http://www.tpex.org.tw/openapi/v1/tpex_mainboard_peratio_analysis'
    json_data = requests.get(link).json()
    df = pd.DataFrame.from_records(json_data)
    return df
    
def get_otc_stock_info(df, stock):
    target_data = df[df['SecuritiesCompanyCode'] == stock]
    name = target_data.iloc[0]['CompanyName']
    priceEarningRatio = target_data.iloc[0]['PriceEarningRatio']
    dividendPerShare = target_data.iloc[0]['DividendPerShare']
    yieldRatio = target_data.iloc[0]['YieldRatio']
    priceBookRatio = target_data.iloc[0]['PriceBookRatio']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_time_input(yy, mm, dd, days = 100):
    time_list = []
    month_num = days//20
    for i in range(month_num):
        yyy = yy
        mmm = mm
        if (mm - i > 0):
            yyy = yy
            mmm = mm - i
        else:
            yyy = yy-1
            mmm = 12 - (i - mm)
        time_list.append([yyy, mmm])
    time_list.reverse()
    return time_list, [yy, mm, dd]

In [5]:
def string_with_comma_to_int(x):
    return int(x.replace(",", ""))

def string_with_comma_to_float(x):
    try:
        return float(x.replace(",", ""))
    except:
        return 0
    
def string_to_float(x):
    try:
        return float(x)
    except:
        return 0
    
def vol_for_twse(x):
    try:
        return round(float(x.replace(",", ""))/1000)
    except:
        return 0

def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

def get_stock_volumn_price(yy, mm, dd, stock_tag, last_date):
    date_tag = str(yy) + str(mm).zfill(2) + "01"
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=open_data&date=%s&stockNo=%s'%(date_tag, stock_tag)
    try:
        df = pd.read_csv(url, encoding='utf_8_sig')
    except:
        return None
    # ["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"]
    df_target = df[['日期', '成交股數', '收盤價', '最高價']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(vol_for_twse) # volumn, 成交張數 = 成交股數 / 1000
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(string_to_float) # price
    df_target.iloc[:, 3] = df_target.iloc[:, 3].apply(string_to_float) # price
    df_target.rename(columns = {'成交股數':'成交張數'}, inplace = True)
    return df_target

def get_otc_stock_volumn_price(yy, mm, dd, stock_tag, last_date):
    # yy = yy - 1911
    date_tag = str(yy) + "%2F" + str(mm).zfill(2) + "%2F01"
    # url = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?d=%s/%s/%s&stkno=%s'%(yy, mm, dd, stock_tag)
    url = 'https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=%s&date=%s&id=&response=open_data'%(stock_tag, date_tag)
    # print(url)
    json_data = requests.get(url).json()
    # ['日期', '成交張數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
    columns = ['日期', '成交張數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
    df = pd.DataFrame(json_data['tables'][0]['data'], columns=columns)
    df_target = df[['日期', '成交張數', '收盤價', '最高價']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(string_with_comma_to_float) # volumn
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(string_with_comma_to_float) # price
    df_target.iloc[:, 3] = df_target.iloc[:, 3].apply(string_with_comma_to_float) # price
    return df_target

def print_log(ss, is_debug):
    if is_debug:
        print(ss)

In [6]:
def get_time_duration_stock_info(time_data, stock_tag, min_volumn = 150, ma_num = 5, isOtc = False):
    df_all = pd.DataFrame()
    time_list, last_date = get_time_input(time_data[0], time_data[1], time_data[2])
    for time_item in time_list:
        year = time_item[0]
        month = time_item[1]
        if isOtc:
            df = get_otc_stock_volumn_price(year, month, "01", stock_tag, last_date)
        else:
            df = get_stock_volumn_price(year, month, "01", stock_tag, last_date)
        if df is None:
            continue
        time.sleep(0.10)
        df_all = pd.concat([df_all, df], axis=0)
        # yy = time_item[0]
        # for mm in range(time_item[1], time_item[2] + 1):
        #     if isOtc:
        #         df = get_otc_stock_volumn_price(yy, mm, "01", stock_tag)
        #     else:
        #         df = get_stock_volumn_price(yy, mm, "01", stock_tag)
        #         if df is None:
        #             continue
        #         time.sleep(0.15)
        #     df_all = pd.concat([df_all, df], axis=0)
    
    # drop out of range date
    df_all = df_all.reset_index()
    df_all.drop(df_all.columns[0], axis=1, inplace = True)
    drop_num = 0
    for i in range(len(df_all)):
        try:
            row_idx = len(df_all) - i - 1
            row_date =df_all.loc[row_idx, "日期"]
            row_date = row_date.replace('*', "")
            row_yy = int(row_date.split('/')[0]) + 1911
            row_mm = int(row_date.split('/')[1])
            row_dd = int(row_date.split('/')[2])
            if row_yy >= last_date[0] and row_mm >= last_date[1] and row_dd > last_date[2]:
                drop_num = drop_num + 1
        except:
            # print(row_date)
            continue
    df_all.drop(df_all.tail(drop_num).index, inplace = True)
      
    df_np = df_all.to_numpy().copy()
    
    if len(df_np) == 0:
        return None, None, None, None
    # if df_np[:, 1].max() > min_volumn:
    #     df_np[:, 1][df_np[:, 1] < min_volumn] = min_volumn
    # if df_np[:, 1].max() == df_np[:, 1].min():
    #     return None, None
    # if df_np[:, 2].max() == df_np[:, 2].min():
    #     return None, None
    
    # df_np[:, 1] = (df_np[:, 1] - df_np[:, 1].min()) / ((df_np[:, 1].max() - df_np[:, 1].min()))
    # df_np[:, 2] = (df_np[:, 2] - df_np[:, 2].min()) / ((df_np[:, 2].max() - df_np[:, 2].min()))
    
    # df_vol_ma = moving_average(df_np[:, 1], ma_num)
    # df_pri_ma = moving_average(df_np[:, 2], ma_num)
    df_vol_ma = df_np[:, 1]
    df_pri_ma = df_np[:, 2]
    df_pri_max_ma = df_np[:, 3]
    
    vol_data = df_vol_ma
    pri_data = df_pri_ma
    return vol_data, pri_data, df_pri_max_ma, df_all

def calculate_target(vol_data, pri_data):
    aa = -5
    bb = -30
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    try:
        # vol_valid = vol_data[aa:].mean() / vol_data[bb:aa].mean() > 5
        # vol_valid = np.median(vol_data[aa:]) / np.median(vol_data[bb:aa]) > 7
        vol_med_factor = 0.5
        vol_cur_factor = 0.75
        vol_max_factor = 0.3
        vol_short = ((1 - vol_cur_factor) * (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:])) + vol_cur_factor * vol_data[-1]) * (1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[aa:]))
        # vol_short = (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:]))
        vol_long = (vol_med_factor * np.median(vol_data[bb:aa]) + (1 - vol_med_factor) * np.mean(vol_data[bb:aa]))*(1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[bb:aa]))
        vol_valid = vol_short / vol_long > 2.8
        vol_valid = vol_valid and (vol_short > 600) and vol_long < 1500
        # previous_high_valid = pri_data[-10:].max() / pri_data[-1] <= 1.08
        # previous_high_valid = previous_high_valid & (pri_data[-20:].max() / pri_data[-1] < 1.05)
        # pri_valid = pri_data[aa:].mean() / pri_data[bb:aa].mean() > 1.03
        # pri_ma = moving_average(pri_data, 4)
        # trend_valid =  pri_ma[-8:].mean()/pri_ma[-88:-80].mean() > 0.8
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        ma_pos_valid =  pri_data[-1] > pri_60ma[-1] and pri_data[-1] > pri_20ma[-1] and pri_data[-1] > pri_10ma[-1] and pri_data[-1] > pri_5ma[-1]
        # trend_valid = (pri_5ma[-1]/pri_10ma[-1] > 0.98) and (pri_5ma[-1]/pri_20ma[-1] > 1.01) and (pri_5ma[-1]/pri_60ma[-1] > 1.02)
        # ma_pos_valid = (pri_data[-1] / pri_5ma[-1] >= 0.95) and (pri_data[-1] / pri_10ma[-1] >= 0.98) and (pri_data[-1] / pri_20ma[-1] >= 1.00) and (pri_data[-1] / pri_60ma[-1] >= 1.00)
        # ma_close_valid = (pri_5ma[-1]) / (pri_10ma[-1]) <= 1.08 and (pri_5ma[-1]) / (pri_10ma[-1]) > 0.94
        # ma_close_valid = ma_close_valid and (pri_10ma[-1]) / (pri_20ma[-1]) <= 1.07 and (pri_10ma[-1]) / (pri_20ma[-1]) > 0.95
    except:
        return False
    # print(vol_valid, trend_valid, ma_pos_valid)
    return vol_valid and ma_pos_valid

def calculate_target_low(vol_data, pri_data):
    aa = -5
    bb = -20
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    try:
        # vol_valid = vol_data[aa:].mean() / vol_data[bb:aa].mean() > 5
        # vol_valid = np.median(vol_data[aa:]) / np.median(vol_data[bb:aa]) > 7
        vol_med_factor = 0.5
        vol_cur_factor = 0.5
        vol_max_factor = 0.3
        vol_short = ((1 - vol_cur_factor) * (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:])) + vol_cur_factor * vol_data[-1]) * (1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[aa:]))
        # vol_short = (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:]))
        vol_long = (vol_med_factor * np.median(vol_data[bb:aa]) + (1 - vol_med_factor) * np.mean(vol_data[bb:aa]))*(1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[bb:aa]))
        vol_valid = vol_short / vol_long > 3
        vol_valid = vol_valid and (vol_short > 1000)
        # previous_high_valid = pri_data[-10:].max() / pri_data[-1] <= 1.08
        # previous_high_valid = previous_high_valid & (pri_data[-20:].max() / pri_data[-1] < 1.05)
        # pri_valid = pri_data[aa:].mean() / pri_data[bb:aa].mean() > 1.03
        # pri_ma = moving_average(pri_data, 4)
        # trend_valid =  pri_ma[-8:].mean()/pri_ma[-88:-80].mean() > 0.8
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        # trend_valid = (pri_5ma[-1]/pri_10ma[-1] > 0.98) and (pri_5ma[-1]/pri_20ma[-1] > 1.01) and (pri_5ma[-1]/pri_60ma[-1] > 1.02)
        # ma_pos_valid = (pri_data[-1] / pri_5ma[-1] >= 0.95) and (pri_data[-1] / pri_10ma[-1] >= 0.98) and (pri_data[-1] / pri_20ma[-1] >= 1.00) and (pri_data[-1] / pri_60ma[-1] >= 1.00)
        # ma_close_valid = (pri_5ma[-1]) / (pri_10ma[-1]) <= 1.08 and (pri_5ma[-1]) / (pri_10ma[-1]) > 0.94
        # ma_close_valid = ma_close_valid and (pri_10ma[-1]) / (pri_20ma[-1]) <= 1.07 and (pri_10ma[-1]) / (pri_20ma[-1]) > 0.95
        ma_pos_valid = pri_10ma[-10] / pri_10ma[-1] <= 1.08
        ma_pos_valid = ma_pos_valid and pri_20ma[-10] / pri_20ma[-1] <= 1.06
        ma_pos_valid = ma_pos_valid and pri_60ma[-10] / pri_60ma[-1] <= 1.05
        ma_pos_valid = ma_pos_valid and pri_data[-1] / pri_60ma[-10] <= 1.05
    except:
        return False
    # print(vol_valid, trend_valid, ma_pos_valid)
    return vol_valid and ma_pos_valid

def calculate_target_long(vol_data, pri_data, pri_max_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -5
    bb = -20
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    smooth_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:]) > 450
        vol_med_factor = 0.5
        vol_cur_factor = 0.5
        vol_max_factor = 0.3
        vol_short = ((1 - vol_cur_factor) * (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:])) + vol_cur_factor * vol_data[-1]) * (1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[aa:]))
        vol_long = (vol_med_factor * np.median(vol_data[bb:aa]) + (1 - vol_med_factor) * np.mean(vol_data[bb:aa]))*(1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[bb:aa]))
        vol_valid = vol_valid and vol_short/vol_long > 0
        
        if not vol_valid:
            print_log("vol invalid", DEBUG)
            return False
        
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        pos_valid = pos_valid and pri_cur[-15:-3].mean() / pri_60ma[-15:-3].mean() <= 1
        pos_valid = pos_valid and pri_cur[-3:].mean() / pri_60ma[-3:].mean() >= 1
    except:
        # print("?")
        return False
    # print(vol_valid , pos_valid , trend_valid)
    return vol_valid and pos_valid and trend_valid and smooth_valid

def calculate_target_trend(vol_data, pri_data, pri_max_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -7
    bb = -28
    if len(vol_data[bb:aa]) == 0 or len(pri_data[bb:aa]) == 0:
        return False
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    smooth_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:]) > 450
        
        if not vol_valid:
            print_log("vol invalid", DEBUG)
            return False
        
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.03 and pri_cur / pri_5ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.983
        print_log("pri_cur / pri_5ma[-1]: %f"%(pri_cur / pri_5ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.03 and pri_cur / pri_10ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.989
        print_log("pri_cur / pri_10ma[-1]: %f"%(pri_cur / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.04 and pri_5ma[-1] / pri_10ma[-1] >= 0.975
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.974
        pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.98
        print_log("pri_5ma[-1] / pri_10ma[-1]: %f"%(pri_5ma[-1] / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.06 and pri_10ma[-1] / pri_20ma[-1] >= 0.98
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.045 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.09 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        print_log("pri_10ma[-1] / pri_20ma[-1]: %f"%(pri_10ma[-1] / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.07 and pri_cur / pri_20ma[-1] >= 0.99
        pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.12 and pri_cur / pri_20ma[-1] >= 0.995
        print_log("pri_cur / pri_20ma[-1]: %f"%(pri_cur / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.04
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.022
        pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.027
        print_log("pri_10ma[-1] / pri_60ma[-1]: %f"%(pri_10ma[-1] / pri_60ma[-1]), DEBUG and not pos_valid)
        
        if not pos_valid:
            print_log("pos invalid", DEBUG)
            return False
        
        trend_valid = trend_valid and pri_10ma[-1] / pri_10ma[-5] >= 0.985
        print_log("pri_10ma[-1] / pri_10ma[-5]: %f"%(pri_10ma[-1] / pri_10ma[-5]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.985
        trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.992
        print_log("pri_20ma[-1] / pri_20ma[-8]: %f"%(pri_20ma[-1] / pri_20ma[-8]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.035
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.023
        trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.021
        print_log("pri_60ma[-1] / pri_60ma[-10]: %f"%(pri_60ma[-1] / pri_60ma[-10]), DEBUG and not trend_valid)
        
        if not trend_valid:
            print_log("trend invalid", DEBUG)
            return False
        
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-2] > 0.955
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-8:].max() > 0.96
        smooth_valid = smooth_valid and pri_data[-1] / pri_max_data[-9:].max() > 0.94
        print_log("pri_data[-1] / pri_max_data[-10:]: %f"%(pri_data[-1] / pri_max_data[-8:].max()), DEBUG and not smooth_valid)
        
        if not smooth_valid:
            print_log("smooth invalid", DEBUG)
            return False
        
        # print(pri_cur / pri_5ma[-1], pri_cur / pri_10ma[-1], pri_5ma[-1] / pri_10ma[-1], pri_10ma[-1] / pri_20ma[-1], pri_cur / pri_20ma[-1] ,pri_10ma[-1] / pri_60ma[-1])
        # print(pri_20ma[-1] / pri_20ma[-8], pri_60ma[-1] / pri_60ma[-10])
    except:
        # print("?")
        return False
    # print(vol_valid , pos_valid , trend_valid)
    return vol_valid and pos_valid and trend_valid and smooth_valid

In [7]:
def calculate_target_bear_trend(vol_data, pri_data, pri_max_data):
    data_min_num = 70
    if len(vol_data) < data_min_num or len(pri_data) < data_min_num:
        # print("Not enough data amount")
        return False
    aa = -7
    bb = -28
    if len(vol_data[bb:aa]) == 0 or len(pri_data[bb:aa]) == 0:
        return False
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    
    pos_valid = True
    vol_valid = True
    trend_valid = True
    smooth_valid = True
    
    try:
        vol_valid = vol_valid and np.max(vol_data[aa:]) > 450
        
        if not vol_valid:
            print_log("vol invalid", DEBUG)
            return False
        
        pri_cur = pri_data[-1]
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.04 and pri_cur / pri_5ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.03 and pri_cur / pri_5ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_5ma[-1] <= 1.017 and pri_cur / pri_5ma[-1] >= 0.96
        print_log("pri_cur / pri_5ma[-1]: %f"%(pri_cur / pri_5ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.04 and pri_cur / pri_10ma[-1] >= 0.97
        # pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.03 and pri_cur / pri_10ma[-1] >= 0.97
        pos_valid = pos_valid and pri_cur / pri_10ma[-1] <= 1.011 and pri_cur / pri_10ma[-1] >= 0.96
        print_log("pri_cur / pri_10ma[-1]: %f"%(pri_cur / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.04 and pri_5ma[-1] / pri_10ma[-1] >= 0.975
        # pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.032 and pri_5ma[-1] / pri_10ma[-1] >= 0.974
        pos_valid = pos_valid and pri_5ma[-1] / pri_10ma[-1] <= 1.02 and pri_5ma[-1] / pri_10ma[-1] >= 0.968
        print_log("pri_5ma[-1] / pri_10ma[-1]: %f"%(pri_5ma[-1] / pri_10ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.06 and pri_10ma[-1] / pri_20ma[-1] >= 0.98
        # pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.045 and pri_10ma[-1] / pri_20ma[-1] >= 0.974
        pos_valid = pos_valid and pri_10ma[-1] / pri_20ma[-1] <= 1.026 and pri_10ma[-1] / pri_20ma[-1] >= 0.91
        print_log("pri_10ma[-1] / pri_20ma[-1]: %f"%(pri_10ma[-1] / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.07 and pri_cur / pri_20ma[-1] >= 0.99
        pos_valid = pos_valid and pri_cur / pri_20ma[-1] <= 1.005 and pri_cur / pri_20ma[-1] >= 0.88
        print_log("pri_cur / pri_20ma[-1]: %f"%(pri_cur / pri_20ma[-1]), DEBUG and not pos_valid)
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.04
        # pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] >= 1.022
        pos_valid = pos_valid and pri_10ma[-1] / pri_60ma[-1] <= 0.913
        print_log("pri_10ma[-1] / pri_60ma[-1]: %f"%(pri_10ma[-1] / pri_60ma[-1]), DEBUG and not pos_valid)
        
        if not pos_valid:
            print_log("pos invalid", DEBUG)
            return False
        
        trend_valid = trend_valid and pri_10ma[-1] / pri_10ma[-5] <= 1.015
        print_log("pri_10ma[-1] / pri_10ma[-5]: %f"%(pri_10ma[-1] / pri_10ma[-5]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] >= 0.985
        trend_valid = trend_valid and pri_20ma[-1] / pri_20ma[-8] <= 1.008
        print_log("pri_20ma[-1] / pri_20ma[-8]: %f"%(pri_20ma[-1] / pri_20ma[-8]), DEBUG and not trend_valid)
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.035
        # trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] >= 1.023
        trend_valid = trend_valid and pri_60ma[-1] / pri_60ma[-10] <= 0.919
        print_log("pri_60ma[-1] / pri_60ma[-10]: %f"%(pri_60ma[-1] / pri_60ma[-10]), DEBUG and not trend_valid)
        
        if not trend_valid:
            print_log("trend invalid", DEBUG)
            return False
        
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-2] < 1.045
        smooth_valid = smooth_valid and pri_data[-1] / pri_data[-8:].max() < 1.04
        smooth_valid = smooth_valid and pri_data[-1] / pri_max_data[-9:].max() < 1.06
        print_log("pri_data[-1] / pri_max_data[-10:]: %f"%(pri_data[-1] / pri_max_data[-8:].max()), DEBUG and not smooth_valid)
        
        if not smooth_valid:
            print_log("smooth invalid", DEBUG)
            return False
        
        # print(pri_cur / pri_5ma[-1], pri_cur / pri_10ma[-1], pri_5ma[-1] / pri_10ma[-1], pri_10ma[-1] / pri_20ma[-1], pri_cur / pri_20ma[-1] ,pri_10ma[-1] / pri_60ma[-1])
        # print(pri_20ma[-1] / pri_20ma[-8], pri_60ma[-1] / pri_60ma[-10])
    except:
        # print("?")
        return False
    # print(vol_valid , pos_valid , trend_valid)
    return vol_valid and pos_valid and trend_valid and smooth_valid

In [8]:
def calculate_target_strong(vol_data, pri_data):
    aa = -5
    bb = -23
    if len(vol_data[bb:aa]) == 0 or len(pri_data[bb:aa]) == 0:
        return False
    if vol_data[bb:aa].mean() == 0:
        return False
    if pri_data[bb:aa].mean() == 0:
        return False
    try:
        # vol_valid = vol_data[aa:].mean() / vol_data[bb:aa].mean() > 5
        # vol_valid = np.median(vol_data[aa:]) / np.median(vol_data[bb:aa]) > 7
        vol_med_factor = 0.5
        vol_cur_factor = 0.5
        vol_max_factor = 0.3
        vol_short = ((1 - vol_cur_factor) * (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:])) + vol_cur_factor * vol_data[-1]) * (1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[aa:]))
        # vol_short = (vol_med_factor * np.median(vol_data[aa:]) + (1 - vol_med_factor) * np.mean(vol_data[aa:]))
        vol_long = (vol_med_factor * np.median(vol_data[bb:aa]) + (1 - vol_med_factor) * np.mean(vol_data[bb:aa]))*(1 - vol_max_factor) + (vol_max_factor * np.max(vol_data[bb:aa]))
        vol_valid = vol_short / vol_long > 4
        vol_valid = vol_valid and vol_long < 5000
        vol_valid = vol_valid and (vol_short > 700)
        # print(vol_short / vol_long, vol_short, vol_long)
        # previous_high_valid = pri_data[-10:].max() / pri_data[-1] <= 1.08
        # previous_high_valid = previous_high_valid & (pri_data[-20:].max() / pri_data[-1] < 1.05)
        # pri_valid = pri_data[aa:].mean() / pri_data[bb:aa].mean() > 1.03
        # pri_ma = moving_average(pri_data, 4)
        # trend_valid =  pri_ma[-8:].mean()/pri_ma[-88:-80].mean() > 0.8
        pri_5ma = moving_average(pri_data, 5)
        pri_10ma = moving_average(pri_data, 10)
        pri_20ma = moving_average(pri_data, 20)
        pri_60ma = moving_average(pri_data, 60)
        # trend_valid = (pri_5ma[-1]/pri_10ma[-1] > 0.98) and (pri_5ma[-1]/pri_20ma[-1] > 1.01) and (pri_5ma[-1]/pri_60ma[-1] > 1.02)
        # ma_pos_valid = (pri_data[-1] / pri_5ma[-1] >= 0.95) and (pri_data[-1] / pri_10ma[-1] >= 0.98) and (pri_data[-1] / pri_20ma[-1] >= 1.00) and (pri_data[-1] / pri_60ma[-1] >= 1.00)
        # ma_close_valid = (pri_5ma[-1]) / (pri_10ma[-1]) <= 1.08 and (pri_5ma[-1]) / (pri_10ma[-1]) > 0.94
        # ma_close_valid = ma_close_valid and (pri_10ma[-1]) / (pri_20ma[-1]) <= 1.07 and (pri_10ma[-1]) / (pri_20ma[-1]) > 0.95
        ma_pos_valid = pri_10ma[-10] / pri_10ma[-1] <= 1.08
        # print(ma_pos_valid)
        ma_pos_valid = ma_pos_valid and pri_20ma[-10] / pri_20ma[-1] <= 1.06
        # print(ma_pos_valid)
        ma_pos_valid = ma_pos_valid and pri_60ma[-10] / pri_60ma[-1] <= 1.05
        # print(ma_pos_valid)
        ma_pos_valid = ma_pos_valid and pri_data[-1] / pri_60ma[-10] <= 1.1
        # print(ma_pos_valid)
    except:
        return False
    # print(vol_valid, ma_pos_valid)
    return vol_valid and ma_pos_valid

In [9]:
def analyze_stock(time_data, isOtc, target_list, thread_total = 1, thread_idx = 0):
    if isOtc:
        df = get_otc_stock_db_info()
        data_path = r".\db\otc.csv"
    else:
        df = get_twse_stock_db_info()
        data_path = r".\db\twse.csv"

    stock_list = []
    with open(data_path, newline='', encoding='utf_8_sig') as csvfile:
        line_list = csv.reader(csvfile)
        for line in line_list:
            stock_list.append(line[0])
    
    stock_list_len = len(stock_list)
    thread_item_num = np.ceil(stock_list_len / thread_total)
    idx_start = thread_item_num * thread_idx
    idx_end = idx_start + thread_item_num
    if thread_idx == thread_total - 1: # last thread
        idx_end = stock_list_len
    
    # target_list = []

    for stock in stock_list[int(idx_start):int(idx_end)]:
        vol_data, pri_data, pri_max_data, df_all = get_time_duration_stock_info(time_data, stock, min_volumn=150, ma_num=1, isOtc=isOtc)
        if vol_data is None or pri_data is None:
            continue
        # valid = calculate_target(vol_data, pri_data)
        # valid = calculate_target_low(vol_data, pri_data)
        # valid = calculate_target_strong(vol_data, pri_data)
        # valid = calculate_target_long(vol_data, pri_data, pri_max_data)
        valid = calculate_target_trend(vol_data, pri_data, pri_max_data)
        # valid = calculate_target_bear_trend(vol_data, pri_data, pri_max_data)
        if valid:
            if isOtc:
                name, priceEarningRatio, yieldRatio, priceBookRatio = get_otc_stock_info(df, stock)
                target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 1])
                print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 1)
            else:
                name, priceEarningRatio, yieldRatio, priceBookRatio = get_twse_stock_info(df, stock)
                target_list.append([stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 0])
                print(stock, name, priceEarningRatio, yieldRatio, priceBookRatio, pri_data[-1], 0)
    # return target_list

In [10]:
THREAD_NUM = 6
threads = []
target_list = []
for i in range(THREAD_NUM):
    threads.append(threading.Thread(target = analyze_stock, args = (time_data, False, target_list, THREAD_NUM, i)))
    threads[i].start()
    
for i in range(THREAD_NUM):
  threads[i].join()

print("Done.")

4571 鈞興-KY 22.31 1.9 3.16 169.0 0
4935 茂林-KY 19.97 1.98 1.17 75.2 0
2603 長榮 6.32 4.68 0.93 206.0 0
6224 聚鼎 46.17 2.18 2.27 77.8 0
2357 華碩 13.42 2.92 1.65 595.0 0
6415 矽力*-KY 152.12 0.39 5.87 528.0 0
9935 慶豐富 20.26 1.43 2.49 34.5 0
9938 百和 22.63 1.25 1.93 81.4 0
Done.


In [11]:
THREAD_NUM = 6
threads = []
target_list_otc = []
for i in range(THREAD_NUM):
    threads.append(threading.Thread(target = analyze_stock, args = (time_data, True, target_list_otc, THREAD_NUM, i)))
    threads[i].start()
    
for i in range(THREAD_NUM):
  threads[i].join()

print("Done.")

1565 精華 18.91 3.28 1.76 211.5 1
8059 凱碩 N/A 0.00 3.59 36.2 1
4931 新盛力 27.42 2.32 2.09 36.3 1
8937 合騏 84.02 0.00 8.70 83.9 1
6220 岳豐 N/A 2.95 1.29 27.0 1
Done.


In [35]:
THREAD_NUM = 6
threads = []
target_list = []
for i in range(THREAD_NUM):
    threads.append(threading.Thread(target = analyze_stock, args = (time_data, False, target_list, THREAD_NUM, i)))
    threads[i].start()
    
for i in range(THREAD_NUM):
  threads[i].join()

print("Done.")

1736 喬山 38.95 0.66 4.73 151.5 0
3013 晟銘電 78.5 0.24 8.76 162.5 0
6768 志強-KY 22.5 3.19 1.66 119.0 0
5434 崇越 17.81 3.18 3.57 314.5 0
2603 長榮 6.09 4.85 0.9 205.5 0
1464 得力 33.47 1.52 1.28 16.4 0
2360 致茂 40.14 1.59 8.1 415.5 0
6176 瑞儀 16.24 4.73 2.88 211.5 0
9938 百和 22.82 1.24 1.95 80.8 0
Done.


In [36]:
THREAD_NUM = 6
threads = []
target_list_otc = []
for i in range(THREAD_NUM):
    threads.append(threading.Thread(target = analyze_stock, args = (time_data, True, target_list_otc, THREAD_NUM, i)))
    threads[i].start()
    
for i in range(THREAD_NUM):
  threads[i].join()

print("Done.")

8069 元太 50.16 1.47 6.83 306.0 1
6220 岳豐 N/A 2.96 1.29 27.0 1
Done.


In [ ]:
if os.path.exists(output_file_path):
    print("File exist!!!")
else:
    with open(output_file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows([time_data])
        writer.writerows([['ID', 'Name', 'PE', 'Yield', 'PB', 'Price', 'isOTC']])
        writer.writerows(target_list)
        writer.writerows(target_list_otc)

In [ ]:
# target_list = analyze_stock(time_data, isOtc = False)
# target_list_otc = analyze_stock(time_data, isOtc = True)

In [ ]:
DEBUG = True
target_list = []
time_data = [2024, 9, 26]

vol_data, pri_data, pri_max_data, df = get_time_duration_stock_info(time_data, 1595, min_volumn=150, ma_num=1, isOtc=1)
# valid = calculate_target(vol_data, pri_data)
valid = calculate_target_strong(vol_data, pri_data)
# valid = calculate_target_trend(vol_data, pri_data, pri_max_data)
print(valid)

True


In [ ]:
1.0928

1.0928